In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import train_test_split

In [16]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
df_mediated_norm = pd.read_csv('../datasets/mediated_schema/mediated_schema_normalized.csv', 
                               dtype={'id_source_vehicles': 'object'})

/tmp/ipykernel_50404/2448643406.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mediated_norm = pd.read_csv('../datasets/mediated_schema/mediated_schema_normalized.csv',


In [4]:
df_mediated_norm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3255536 entries, 0 to 3255535
Data columns (total 19 columns):
 #   Column               Dtype  
---  ------               -----  
 0   id_source_vehicles   object 
 1   id_source_used_cars  object 
 2   location             object 
 3   price                float64
 4   year                 float64
 5   manufacturer         object 
 6   model                object 
 7   cylinders            object 
 8   fuel_type            object 
 9   mileage              float64
 10  transmission         object 
 11  vin                  object 
 12  traction             object 
 13  body_type            object 
 14  main_color           object 
 15  description          object 
 16  latitude             float64
 17  longitude            float64
 18  pubblication_date    object 
dtypes: float64(5), object(14)
memory usage: 471.9+ MB


In [7]:
df_mediated_norm.head()

,id_source_vehicles,id_source_used_cars,location,price,year,manufacturer,model,cylinders,fuel_type,mileage,transmission,vin,traction,body_type,main_color,description,latitude,longitude,pubblication_date
0,7316814884,NaN,auburn,33590.0,2014.0,gmc,sierra 1500 crew cab slt,8 cylinders,gas,57923.0,other,3GTP1VEC4EG551563,NaN,pickup,white,carvana is the safer way to buy a car during t...,32.590,-85.4800,2021-05-04T12:31:18-0500
1,7316814758,NaN,auburn,22590.0,2010.0,chevrolet,silverado 1500,8 cylinders,gas,71229.0,other,1GCSCSE06AZ123805,NaN,pickup,blue,carvana is the safer way to buy a car during t...,32.590,-85.4800,2021-05-04T12:31:08-0500
2,7316814989,NaN,auburn,39590.0,2020.0,chevrolet,silverado 1500 crew,8 cylinders,gas,19160.0,other,3GCPWCED5LG130317,NaN,pickup,red,carvana is the safer way to buy a car during t...,32.590,-85.4800,2021-05-04T12:31:25-0500
3,7316743432,NaN,auburn,30990.0,2017.0,toyota,tundra double cab sr,8 cylinders,gas,41124.0,other,5TFRM5F17HX120972,NaN,pickup,red,carvana is the safer way to buy a car during t...,32.590,-85.4800,2021-05-04T10:41:31-0500
4,7316356412,NaN,auburn,15000.0,2013.0,ford,f150 xlt,6 cylinders,gas,128000.0,automatic,NaN,rwd,truck,black,2013 f 150 xlt v6 4 door good condition leveli...,32.592,-85.5189,2021-05-03T14:02:03-0500


In [8]:
# Pulizia dei record con valore VIN non corretti, per la generazione della Ground Truth

def is_valid_vin_checksum(vin):
    """
    Algoritmo ufficiale per la validazione della 9a cifra (Check Digit) del VIN.
    """
    if not isinstance(vin, str) or len(vin) != 17:
        return False
    
    # Mappa valori caratteri ISO
    val_map = {
        'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'H':8,
        'J':1, 'K':2, 'L':3, 'M':4, 'N':5, 'P':7, 'R':9, 'S':2,
        'T':3, 'U':4, 'V':5, 'W':6, 'X':7, 'Y':8, 'Z':9,
        '0':0, '1':1, '2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9
    }
    # Pesi per ogni posizione
    weights = [8, 7, 6, 5, 4, 3, 2, 10, 0, 9, 8, 7, 6, 5, 4, 3, 2]
    
    try:
        total = sum(val_map[vin[i]] * weights[i] for i in range(17) if i != 8)
        remainder = total % 11
        check_digit = 'X' if remainder == 10 else str(remainder)
        return vin[8] == check_digit
    except KeyError:
        return False

def count_differences(row1, row2, colonne_da_confrontare):
    diff = 0
    for col in colonne_da_confrontare:
        v1, v2 = row1[col], row2[col]
        if pd.isna(v1) and pd.isna(v2):
            continue
        if v1 != v2:
            diff += 1
    return diff

def deduplica_vin_per_similarita(
    df,
    vin_col='vin',
    date_col='pubblication_date',
    max_diff=3
):
    colonne_da_confrontare = [
        c for c in df.columns
        if c not in {vin_col, date_col, 'id_source_vehicles', 'id_source_used_cars', 'description'}
    ]

    # Convertiamo temporaneamente in una colonna di supporto per l'ordinamento
    # senza sovrascrivere quella originale (che vogliamo preservare come stringa)
    df['temp_date_sort'] = pd.to_datetime(df[date_col], errors='coerce')

    record_finali = []

    for vin, gruppo in df.groupby(vin_col):
        if len(gruppo) == 1:
            record_finali.append(gruppo.iloc[0])
            continue

        # Ordiniamo usando la colonna di supporto
        gruppo = gruppo.sort_values(by='temp_date_sort', ascending=False)
        usati = set()

        for i, row_i in gruppo.iterrows():
            if i in usati:
                continue
            cluster = [i]
            for j, row_j in gruppo.iterrows():
                if j == i or j in usati:
                    continue
                diff = count_differences(row_i, row_j, colonne_da_confrontare)
                if diff <= max_diff:
                    cluster.append(j)
            
            for idx in cluster:
                usati.add(idx)
            
            # Teniamo il record del cluster (ha la data originale)
            record_finali.append(gruppo.loc[cluster].iloc[0])

    # Creiamo il DataFrame e rimuoviamo la colonna di supporto
    df_res = pd.DataFrame(record_finali).reset_index(drop=True)
    if 'temp_date_sort' in df_res.columns:
        df_res = df_res.drop(columns=['temp_date_sort'])
    
    return df_res


def pulizia_vin_avanzata(df, colonna_vin='vin', colonna_data='pubblication_date'):
    """
    Esegue pulizia alfanumerica, rimozione null, validazione checksum e deduplicazione.
    """
    # 1. RIMOZIONE RECORD CON VIN NULLI (Fondamentale per la Ground Truth)
    df_clean = df.dropna(subset=[colonna_vin]).copy()
    
    # 2. NORMALIZZAZIONE E PULIZIA STRINGA
    # Rimuove tutto ciò che non è lettera o numero e rende maiuscolo
    df_clean[colonna_vin] = df_clean[colonna_vin].astype(str).str.upper().str.replace(r'[^A-Z0-9]', '', regex=True)
    
    # 3. FILTRI FORMATO E CARATTERI VIETATI (I, O, Q)
    # Lunghezza 17 e controllo caratteri legali
    regex_legale = r'^[A-HJ-NPR-Z0-9]{17}$'
    df_clean = df_clean[df_clean[colonna_vin].str.contains(regex_legale, regex=True)]
    
    # 4. RIMOZIONE PLACEHOLDER (Pattern ripetuti)
    regex_placeholder = r'^(.)\1{16}$|12345678|ABCDEFGH'
    df_clean = df_clean[~df_clean[colonna_vin].str.contains(regex_placeholder, regex=True)]
    
    # 5. VALIDAZIONE CHECKSUM
    # Applichiamo la funzione is_valid_vin_checksum
    df_clean['vin_valido'] = df_clean[colonna_vin].apply(is_valid_vin_checksum)
    df_clean = df_clean[df_clean['vin_valido'] == True].drop(columns=['vin_valido'])
    
    # 6. DEDUPLICAZIONE AVANZATA VIN
    df_clean = deduplica_vin_per_similarita(df_clean, vin_col=colonna_vin, date_col=colonna_data, max_diff=3)
    
    return df_clean

In [9]:
df_mediated_norm['vin'].value_counts()

vin
1FMJU1JT1HEA52352    261
3C6JR6DT3KG560649    236
1FTER1EH1LLA36301    231
5TFTX4CN3EX042751    227
1GCHTCE37G1186784    214
                    ... 
1C4RJFBG4HC908852      1
1FMCU9H99LUB55938      1
KM8J3CA47MU300479      1
1HGCV1F4XLA111931      1
5FNYF6H52MB004143      1
Name: count, Length: 2964967, dtype: int64

In [10]:
# --- APPLICAZIONE ---

# Applichiamo la funzione al dataset normalizzato per la pulizia dei record con VIN non validi
print(f"Record pre-pulizia: Dataset={len(df_mediated_norm)}")

df_sanificato_norm = pulizia_vin_avanzata(df_mediated_norm)

print(f"Record post-pulizia: Dataset={len(df_sanificato_norm)}")

Record pre-pulizia: Dataset=3255536


/tmp/ipykernel_50404/2200145395.py:107: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_clean = df_clean[~df_clean[colonna_vin].str.contains(regex_placeholder, regex=True)]
/tmp/ipykernel_50404/2200145395.py:51: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['temp_date_sort'] = pd.to_datetime(df[date_col], errors='coerce')


Record post-pulizia: Dataset=2964655


In [11]:
df_sanificato_norm['vin'].value_counts()

vin
KNDJP3A55H7463251    5
1FTYR10U61PA73092    5
1FTEW1EP5HFA09219    5
1FTEW1EF0FFB34035    4
1FTNE24L89DA76367    4
                    ..
1N4BL4CV7KC153917    1
1N4BL4CV7KC154274    1
1N4BL4CV7KC154596    1
1N4BL4CV7KC155098    1
1N4BL4CV7KC145302    1
Name: count, Length: 2958730, dtype: int64

In [12]:
def genera_ground_truth(df_mediato, ratio_negativi=2.0, random_state=42):
    """
    Genera GT includendo match A-B, A-A e B-B.
    Formato: (id_A, attr_A, id_B, attr_B, label)
    """
    np.random.seed(random_state)
    
    attributi = [
        'location', 'price', 'year', 'manufacturer', 'model', 'cylinders',
        'fuel_type', 'mileage', 'transmission', 'traction',
        'body_type', 'main_color', 'description',
        'latitude', 'longitude', 'pubblication_date'
    ]

    # 1. Creiamo un ID unico "canonico" per ogni riga, dato che un record
    # ha o l'id_source_vehicles o l'id_source_used_cars.
    df = df_mediato.copy()
    df['id_univoco'] = df['id_source_vehicles'].fillna(df['id_source_used_cars']).astype(str)
    
    # Teniamo solo le colonne necessarie per il join (ID, VIN e Attributi)
    df_per_join = df[['id_univoco', 'vin'] + attributi]

    # =====================================================
    # 2. GENERAZIONE POSITIVI (Label 1) - Includendo A-A, B-B, A-B
    # =====================================================
    # Self-join sul VIN
    positivi = pd.merge(
        df_per_join,
        df_per_join,
        on='vin',
        suffixes=('_A', '_B')
    )

    # REGOLE DI UNIVOCITÀ E FILTRO:
    # 1. Rimuoviamo il confronto di un record con se stesso (id_A == id_B)
    # 2. Per evitare coppie duplicate invertite (r1,r2 e r2,r1), 
    #    teniamo solo dove id_A è lessicalmente minore di id_B.
    positivi = positivi[positivi['id_univoco_A'] < positivi['id_univoco_B']].copy()

    positivi['label'] = 1
    positivi = positivi.rename(columns={'id_univoco_A': 'id_A', 'id_univoco_B': 'id_B'})
    
    # Rimuoviamo il VIN (Punto 4.B)
    positivi = positivi.drop(columns=['vin'])
    
    n_positivi = len(positivi)
    print(f"Coppie positive totali (A-B, A-A, B-B): {n_positivi}")

    # =====================================================
    # 3. GENERAZIONE NEGATIVI (Label 0)
    # =====================================================
    n_negativi_target = int(n_positivi * ratio_negativi)
    negativi_list = []
    
    # Campionamento casuale per generare negativi
    # Usiamo un approccio a blocchi per velocità
    while len(negativi_list) < n_negativi_target:
        idx1 = np.random.choice(df.index, size=n_negativi_target)
        idx2 = np.random.choice(df.index, size=n_negativi_target)
        
        tmp_A = df.loc[idx1, ['id_univoco', 'vin'] + attributi].reset_index(drop=True)
        tmp_B = df.loc[idx2, ['id_univoco', 'vin'] + attributi].reset_index(drop=True)
        
        # Filtro: ID diversi AND VIN diversi AND ordine univoco
        mask = (tmp_A['id_univoco'] != tmp_B['id_univoco']) & \
               (tmp_A['vin'] != tmp_B['vin'])
        
        # Per i negativi applichiamo lo stesso ordinamento per coerenza
        # Se id_A > id_B, invertiamo i record per mantenere id_A < id_B
        swap_mask = tmp_A['id_univoco'] > tmp_B['id_univoco']
        
        # Applichiamo lo swap dove necessario
        tmp_A.loc[swap_mask], tmp_B.loc[swap_mask] = tmp_B.loc[swap_mask], tmp_A.loc[swap_mask]
        
        batch_neg = pd.concat([
            tmp_A.add_suffix('_A'), 
            tmp_B.add_suffix('_B')
        ], axis=1)[mask]
        
        negativi_list.append(batch_neg)
        combined_neg = pd.concat(negativi_list).drop_duplicates(subset=['id_univoco_A', 'id_univoco_B'])
        if len(combined_neg) >= n_negativi_target:
            negativi = combined_neg.head(n_negativi_target).copy()
            break

    negativi = negativi.rename(columns={'id_univoco_A': 'id_A', 'id_univoco_B': 'id_B'})
    negativi = negativi.drop(columns=['vin_A', 'vin_B'])
    negativi['label'] = 0
    
    print(f"Coppie negative generate: {len(negativi)}")

    # =====================================================
    # 4. ASSEMBLAGGIO FINALE
    # =====================================================
    ground_truth = pd.concat([positivi, negativi], ignore_index=True)
    ground_truth = ground_truth.sample(frac=1, random_state=random_state).reset_index(drop=True)

    # Ordinamento finale colonne
    cols_A = ['id_A'] + [f"{a}_A" for a in attributi]
    cols_B = ['id_B'] + [f"{a}_B" for a in attributi]
    ground_truth = ground_truth[cols_A + cols_B + ['label']]

    return ground_truth

In [13]:
# --- ESECUZIONE ---

ground_truth_df = genera_ground_truth(df_sanificato_norm, ratio_negativi=2.0)

# Stampa info finali
print(f"\nShape finale Ground Truth: {ground_truth_df.shape}")
print(f"Distribuzione Label:\n{ground_truth_df['label'].value_counts()}")

Coppie positive totali (A-B, A-A, B-B): 6089
Coppie negative generate: 12178

Shape finale Ground Truth: (18267, 35)
Distribuzione Label:
label
0    12178
1     6089
Name: count, dtype: int64


In [14]:
assert not ground_truth_df.isna().all(axis=1).any(), "Righe completamente vuote!"
assert ground_truth_df.filter(like='_A').isna().all(axis=1).sum() == 0
assert ground_truth_df.filter(like='_B').isna().all(axis=1).sum() == 0

In [23]:
ground_truth_df[ground_truth_df['label']==0].head()

,id_A,location_A,price_A,year_A,manufacturer_A,model_A,cylinders_A,fuel_type_A,mileage_A,transmission_A,traction_A,body_type_A,main_color_A,description_A,latitude_A,longitude_A,pubblication_date_A,id_B,location_B,price_B,year_B,manufacturer_B,model_B,cylinders_B,fuel_type_B,mileage_B,transmission_B,traction_B,body_type_B,main_color_B,description_B,latitude_B,longitude_B,pubblication_date_B,label
0,S2_2650198,sacramento,14495.0,2017.0,hyundai,santa fe sport,i4,gasoline,63795.0,a,fwd,suv crossover,blue,backup camera bluetooth wireless connectabilit...,38.5960,-121.4010,2020-07-09,S2_753415,alexandria,7869.0,2009.0,hyundai,sonata,i4,gasoline,63847.0,a,fwd,sedan,blue,gray leather carfax one owner slate blue metal...,38.7557,-77.0837,2020-08-18,0
1,S2_272859,saco,24345.0,2021.0,subaru,crosstrek,h4,gasoline,1.0,m,awd,suv crossover,silver,additional info standard model,43.5262,-70.4274,2020-09-09,S2_367215,aberdeen,14734.0,2021.0,chevrolet,spark,i4,gasoline,0.0,a,fwd,hatchback,white,additional info summit white seats front high ...,39.5116,-76.1619,2020-09-06,0
2,S2_669800,winchester,17494.0,2020.0,honda,fit,i4,gasoline,2.0,cvt,fwd,hatchback,unknown,this 2020 modern steel metallic honda fit lx f...,39.1158,-78.2001,2020-07-08,S2_806378,roanoke,8890.0,2012.0,honda,civic,i4,gasoline,145367.0,a,fwd,sedan,white,taffeta white 2012 honda civic ex fwd 5 speed ...,37.3463,-79.9568,2020-09-06,0
3,S2_1247647,hawkinsville,32585.0,2020.0,ford,ranger,i4,gasoline,12.0,a,4wd,pickup truck,white,visit way brothers online at to see more pictu...,32.2814,-83.4752,2020-07-07,S2_354749,oakhurst,42860.0,2017.0,lexus,rx 350,v6,gasoline,16014.0,a,awd,suv crossover,gray,l certified pre owned carfax 1 owner f sport w...,40.2717,-74.0426,2020-08-23,0
4,S2_1272673,lagrange,90255.0,2019.0,cadillac,escalade esv,v8,gasoline,4338.0,a,4wd,suv crossover,silver,additional info cadillac user experience with ...,33.0557,-85.0289,2019-05-04,S2_867701,greensboro,24295.0,2020.0,nissan,rogue,i4,gasoline,2.0,cvt,fwd,suv crossover,gray,dealer discount of 3 810 off msrp gun metallic...,36.0600,-79.8765,2020-05-30,0


In [24]:
ground_truth_df[ground_truth_df['label']==1].head()

,id_A,location_A,price_A,year_A,manufacturer_A,model_A,cylinders_A,fuel_type_A,mileage_A,transmission_A,traction_A,body_type_A,main_color_A,description_A,latitude_A,longitude_A,pubblication_date_A,id_B,location_B,price_B,year_B,manufacturer_B,model_B,cylinders_B,fuel_type_B,mileage_B,transmission_B,traction_B,body_type_B,main_color_B,description_B,latitude_B,longitude_B,pubblication_date_B,label
6,7306828517,stockton,9488.0,2015.0,jeep,cherokee,NaN,gas,128574.0,automatic,fwd,suv,black,2015 jeep cherokee latitude 4dr suv call or te...,38.570333,-121.521673,2021-04-14T12:15:35-0700,7316606912,sf bay area,8777.0,2015.0,jeep,cherokee,NaN,gas,128574.0,automatic,fwd,suv,black,2015 jeep cherokee latitude 4dr suv suv ready ...,38.570309,-121.521625,2021-05-03T23:00:15-0700,1
14,7316702251,monroe,24640.0,2013.0,ram,1500,NaN,gas,114000.0,automatic,4wd,pickup,silver,2013 ram 1500 4wd crew cab laramie offered by ...,41.612850,-83.480782,2021-05-04T10:31:13-0400,7316859807,mansfield,0.0,2013.0,ram,1500,NaN,gas,114000.0,automatic,4wd,pickup,silver,2013 ram 1500 4wd crew cab laramie offered by ...,41.621510,-83.493576,2021-05-04T14:41:18-0400,1
18,7315746872,sioux city,0.0,2007.0,chevrolet,avalanche,8 cylinders,gas,148137.0,automatic,4wd,truck,silver,2007 chevrolet avalanche 4wd crew cab 130 lt w...,41.855551,-97.443365,2021-05-02T10:00:55-0500,7316738869,lincoln,12999.0,2007.0,chevrolet,avalanche,8 cylinders,gas,148137.0,automatic,4wd,truck,silver,2007 chevrolet avalanche 4wd crew cab 130 lt w...,41.438366,-97.316899,2021-05-04T10:33:50-0500,1
19,7302068235,jackson,5995.0,2012.0,buick,lacrosse,NaN,gas,139731.0,automatic,NaN,NaN,NaN,we have over 350 vehicles in stock view our en...,43.186723,-84.163862,2021-04-05T11:20:01-0400,7309214489,upper peninsula,6995.0,2012.0,buick,lacrosse,NaN,gas,139731.0,automatic,NaN,NaN,NaN,we have over 500 vehicles in stock see our ent...,43.182400,-84.112200,2021-04-19T10:27:46-0400,1
25,7316331659,denver,9988.0,2012.0,bmw,1series 128i,6 cylinders,gas,87202.0,automatic,rwd,convertible,NaN,price 9 988 price expires 5 8 2021 at 8pmin ho...,39.740343,-105.044341,2021-05-03T12:23:12-0600,S2_2732467,denver,9988.0,2012.0,bmw,1 series,i6,gasoline,87202.0,a,rwd,convertible,blue,powerful fuel efficient engines nimble handlin...,39.740200,-105.044000,2020-06-06,1


In [25]:
ground_truth_df.to_csv('../datasets/ground_truth/ground_truth.csv', index=False)

In [26]:
# Split GT → train vs eval (STRATIFICATO)
GT_train, GT_eval = train_test_split(
    ground_truth_df,
    test_size=0.30,
    random_state=42,
    stratify=ground_truth_df['label']
)

print("Distribuzione GT_train:")
print(GT_train['label'].value_counts(normalize=True))

print("\nDistribuzione GT_eval:")
print(GT_eval['label'].value_counts(normalize=True))

print(f"Split completato:")
print(f"- Train: {len(GT_train)} righe")
print(f"- Eval:   {len(GT_eval)} righe")

Distribuzione GT_train:
label
0    0.666667
1    0.333333
Name: proportion, dtype: float64

Distribuzione GT_eval:
label
0    0.666667
1    0.333333
Name: proportion, dtype: float64
Split completato:
- Train: 12786 righe
- Eval:   5481 righe


In [27]:
# Salvataggio su file CSV
GT_eval.to_csv("../datasets/ground_truth/GT_eval.csv", index=False)

In [28]:
# Eseguiamo uno split stratificato per mantenere la proporzione tra match (1) e non-match (0)
# Primo split: 70% Train e 30% Temporaneo (che divideremo in Val e Test)
train_set, temp_set = train_test_split(
    GT_train, 
    test_size=0.30, 
    random_state=42, 
    stratify=GT_train['label']
)

# Secondo split: Dividiamo il 30% temporaneo a metà (15% Val, 15% Test)
val_set, test_set = train_test_split(
    temp_set, 
    test_size=0.50, 
    random_state=42, 
    stratify=temp_set['label']
)


print(f"Split completato:")
print(f"- Train: {len(train_set)} righe")
print(f"- Val:   {len(val_set)} righe")
print(f"- Test:  {len(test_set)} righe\n")

Split completato:
- Train: 8950 righe
- Val:   1918 righe
- Test:  1918 righe



In [29]:
# Salvataggio su file CSV
train_set.to_csv("../datasets/ground_truth/GT_train/train.csv", index=False)
val_set.to_csv("../datasets/ground_truth/GT_train/val.csv", index=False)
test_set.to_csv("../datasets/ground_truth/GT_train/test.csv", index=False)